#### Soft Attention Mechanism (Bahdanau et al., 2015)

$$\text{Attention Network} = \text{softmax}(V(\tanh(W_1(a)+ W_2(h_0))))$$

Esta ecuación describe un mecanismo de atención, el cual es un componente crucial en muchas arquitecturas de redes neuronales en el procesamiento de lenguaje natural (NLP).  
  
En este notebook, se implementa un mecanismo de atención suave en PyTorch y utilizando solo Numpy para la implementación de este mecanismo/formula.  
  
Los componentes de la ecuación son:

- $a$: es la secuencia de entrada, la cual es una secuencia de vectores de características, tipicamente se obtiene de una capa de encoder en un modelo de secuencia a secuencia.En el contexto de un modelo RNN, $a$ es la secuencia de estados ocultos de la capa de encoder correspondientes a cada token en la secuencia de entrada.

- $h0$: simboliza el estado inicial oculto del decoder. en el proceso iterativo de decodificación, $h0$ es el estado oculto del decoder en el paso de tiempo anterior.

- $W1$ y $W2$: son matrices de peso entrenables. $W1$ es aplicado a las entradas $a$ y $W2$ es aplicado al estado oculto $h0$. Estas matrices transforman sus respectivas entradas en un espacio en común donde pueden ser combinadas para calcular la atención.

- $V$: Otro vector de peso entrenable que es aplicado despues de la adición y transformación no lineal de las anotaciones del encoder y el estado oculto del decoder. Este vector proyecta la combinación en un espacio de atención, donde se puede interpretar como un score de atención no normalizado.

- $\tanh$: es la función de activación tangente hiperbólica. Es una función de activación no lineal usada para introducir la no linealidad en la transformación de las entradas, permitiendo al modelo aprender relaciones complejas.

- $\text{softmax}$: es una función de activación que toma un vector de entrada y lo normaliza en un vector de probabilidades. En el contexto de la atención, el vector de entrada es el score de atención no normalizado, y el vector de salida es el score de atención normalizado.

### Funcionalidad y proposito

El mecanismo de atención le permite al modelo enfocarse en diferentes partes de la secuencia de entrada en cada paso de la secuencia de salida, habilitando más generación de salidas consiente del contexto.  
  
Selecciona dinamicamente que partes de las anotaciones de la entrada son más relevantes para predecir cada token de salida, solucionando la limitante de los vectores de salida de longitud fija en los modelos de secuencia a secuencia tradicionales.  
  
La ecuación calcula el set de pesos de atención, que son usados para producir una suma ponderada de las anotaciones de entrada. Esta suma ponderada se convierte en el vector de contexto para la salida en el paso de tiempo actual, proporcionando entradas personalizadas al decoder basado en que necesita el modelo para predecir lo siguiente.  

### Interpretación técnica

- La operación $W1(a)$ y $W2(h0)$ transforman las anotaciones de entrada y el estado oculto inicial del decoder en un nuevo espacio donde pueden ser comparados. El objetivo es entender que tan relevante es cada parte de la entrada con respecto del estado actual del decoder.  
  
- La adición de estas transformaciones seguidos de la no linearidad $\tanh$ combina estas dos fuentes de información en una representación única que captura ambos contenidos de entrada y el actual enfoque del decoder.  
  
- La operación $V$ y la función de activación $\text{softmax}$ combierten esta representación combinada en un set de pesos de atención. Estos pesos determinan que tanto debería contribuir cada parte de la entrada al paso actual del decoder.  
  
- El mecanismo de atención mejora la habilidad del modelo de capturar dependencias a larga distancia y administrar entradas y salidas de diferente longitud, solucionando retos clave en tareas de modelado de secuencias.  
  
Esta ecuación es un componente fundamental en la implementación del mecanismo de atención en los modelos de redes neuronales, especialmente en el contexto de NLP y tareas de modelado de secuencias.

### A continuación un ejemplo de implementación de este mecanismo en PyTorch

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

class AttentionNetwork(nn.Module):
  def __init__( self, annotation_dim, hidden_dim, attention_dim ):
    """
    Inicializar la red de atención.

    Parametros:
    - annotation_dim: Dimensión de las anotaciones.
    - hidden_dim: Dimensión de la capa oculta.
    - attention_dim: Dimensión de la capa de atención.
    """
    super( AttentionNetwork, self).__init__()

    # Definir las matrices de pesos para la red de atención.
    self.W1 = nn.Linear( annotation_dim, attention_dim, bias = False )
    # Aplica W1 a las anotaciones.
    self.W2 = nn.Linear( hidden_dim, attention_dim, bias = False )
    # Aplica W2 a la capa oculta.
    self.V = nn.Linear( attention_dim, 1, bias = False )
    # Aplica V a la salida de tanh de la suma de W1 y W2.

  def forward( self, annotations, hidden ):
    """
    Forward pass de la red de atención. (Esto solo quiere decir que se aplican las operaciones definidas en el constructor).

    Parametros:
    - annotations: Tensor conteniendo las anotaciones provenientes del encoder ( shape: batch_size x seq_len x annotation_dim ).
    - hidden: Tensor conteniendo la capa oculta actual del decoder ( shape: batch_size x hidden_dim ).

    Retorna:
    - attention_weights: Tensor conteniendo los pesos de atención para cada anotación ( shape: batch_size x seq_len ).
    """

    # Espandir la capa oculta para que tenga la misma forma que las anotaciones para poder sumarlas.
    hidden = hidden.unsqueeze( 1 ).expand_as( annotations )

    # Calcular los pesos de atención.
    attn_scores = self.V( torch.tanh( self.W1( annotations ) + self.W2( hidden ) ) )

    # oprimir la ultima dimensión y aplicar softmax para obtener los pesos de atención.
    attention_weights = F.softmax( attn_scores.squeeze( -1 ), dim = -1 )

    return attention_weights




In [21]:
import torch

# Asumimos que ya se ha definido el encoder y el decoder.
# y que estas son las dimensiones de las anotaciones y la capa oculta.
annotation_dim = 256 # Dimensión de las anotaciones de salida del encoder
hidden_dim = 256 # Dimensión de la capa oculta del decoder
attention_dim = 256 # Dimensión de representación intermedia de la red de atención

# Inicializar la red de atención.
attention_network = AttentionNetwork( annotation_dim, hidden_dim, attention_dim )

# Ejemplo de anotaciones y capa oculta para probar la red de atención.
batch_size = 2 # El valor debe de 
seq_len = 10
annotations = torch.randn( batch_size, seq_len, annotation_dim )
hidden = torch.randn( batch_size, hidden_dim )

# Calcular los pesos de atención.
attention_weights = attention_network( annotations, hidden )

print( f"La forma de los pesos de atencion: {attention_weights.shape}")
print( attention_weights )

La forma de los pesos de atencion: torch.Size([2, 10])
tensor([[0.1254, 0.1272, 0.1006, 0.1343, 0.0925, 0.1244, 0.0942, 0.0652, 0.0572,
         0.0791],
        [0.1020, 0.1035, 0.1168, 0.0840, 0.1316, 0.0810, 0.1046, 0.0723, 0.0937,
         0.1105]], grad_fn=<SoftmaxBackward0>)


### A continuación un ejemplo de implementación de este mecanismo en Numpy

In [22]:
import numpy as np

def softmax(x):
  """ Cálcula los valores softmax por cada set de Scores en x"""
  e_x = np.exp( x - np.max( x, axis = -1, keepdims = True ) )
  return e_x / e_x.sum( axis = -1, keepdims = True )

class AttentionNetworkNumpy:
  def __init__( self, annotation_dim, hidden_dim, attention_dim ):
    """
    Inicializar la red de atención con arreglos Numpy

    Parametros:
    - annotation_dim: Dimensión de las entradas de anotaciones (a)
    - hidden_dim: Dimensión de los estados ocultos del decoder (h_0)
    - attention_dim: Dimensión intermedia, representación del mecanismo de atención
    """

    # Inicializar los pesos con valores aleatorios (semi aleatorios)
    self.W1 = np.random.rand( annotation_dim, attention_dim )
    self.W2 = np.random.rand( hidden_dim, attention_dim )
    self.V = np.random.rand( attention_dim, 1 )

  def forward( self, annotations, hidden ):
    """
    Forward pass através de la red de atención usando Numpy

    Parametros:
    - annotations: Arreglo Numpy con las anotaciones provenientes del Encoder ( shape: batch_size x seq_len x annotation_dim )
    - hidden: Arreglo Numpy con los estados ocultos actuales del decoder ( shape: batch_size x hidden_dim )

    Retorna:
    - attention_weights: Arreglo Numpy con los pesos de atención ( shape: batch_size x seq_len )
    """

    # Expandir los estados ocultos para igualar la dimensión de las anotaciones para la suma de elementos
    hidden_expanded = np.expand_dims( hidden, axis = 1 )
    hidden_expanded = np.tile( hidden_expanded, ( 1, annotations.shape[1], 1 ) )

    # Cálcular los scores de atención
    attn_scores = np.dot( np.tanh( np.dot( annotations, self.W1 ) + np.dot( hidden_expanded, self.W2 ) ), self.V )

    # Comprimir la ultima dimensión y aplicar softmax para obtener los pesos de atención
    attention_weights = softmax( attn_scores.squeeze(-1) )

    return attention_weights


In [50]:
import numpy as np

# Assuming these dimensions for the sake of example
annotation_dim = 256  # Dimension of the encoder's output annotations
hidden_dim = 256      # Dimension of the decoder's hidden state
attention_dim = 128   # Intermediate attention representation dimension

# Initialize the attention network with NumPy
attention_network_numpy = AttentionNetworkNumpy(annotation_dim, hidden_dim, attention_dim)

# Example annotations and hidden state (randomly generated for demonstration)
batch_size = 2
seq_len = 10
annotations = np.random.randn(batch_size, seq_len, annotation_dim)
hidden = np.random.randn(batch_size, hidden_dim)

# Forward pass through the attention network
attention_weights = attention_network_numpy.forward(annotations, hidden)

print("Attention weights shape:", attention_weights.shape)  # Expected shape: (batch_size, seq_len)
print(attention_weights)


Attention weights shape: (2, 10)
[[6.61171973e-01 1.07394587e-03 3.88658447e-30 5.08878893e-42
  4.44135299e-08 6.93621233e-11 1.85656613e-30 1.92369569e-01
  1.45384468e-01 9.98078514e-17]
 [2.03947162e-29 6.12211979e-25 1.48010738e-01 1.64206977e-04
  8.51801214e-01 1.18565787e-19 3.24040040e-13 4.57690152e-08
  4.17307546e-29 2.37950663e-05]]
